In [ ]:
##vscode python 
##********************************************************************--
##author:Razzak, Khandaker
##create time:2022-11-18 23:46:17
##Created Python Logic for Scripted Chatbot.
##********************************************************************--

Creating a Chatbot using word2vec is ideal for any E-commerce or any insitution. This a toy model of the backend methods of a Chatbot.

In [3]:
import re
import numpy as np
import pandas as pd
from numpy import dot
import tensorflow as tf
from numpy.linalg import norm
from itertools import product
from tensorflow import keras

Important Functions that will be used down the line

In [86]:
def combination_of_words(arr1, arr2)->list:
    """This will return a list of the combination of words"""
    return list(product(arr1, arr2))

def consine_similarity(word_1:str, word_2:str)->float:
    """This function return the cosine similarity"""
    word_1_coord = word_ebmedding_dict.get(word_1)
    word_2_coord = word_ebmedding_dict.get(word_2)

    similarity_score = 1 - (dot(word_1_coord, word_2_coord)/(norm(word_1_coord) * norm(word_2_coord)))

    return similarity_score

def sentence_similarity(corpus_sentence:str, test_sentence:str)->float:
    """This function will return the a consine similarity between two sentences"""

    similarity_score_total = 0

    corpus_sentence_list = text_preprocessing(corpus_sentence)

    test_question_list = [a_word for a_word in text_preprocessing(test_sentence) if a_word in albert_dictionary.keys()]

    
    word_pairs_for_similarity = combination_of_words(corpus_sentence_list, test_question_list)
    
    for a_pair in word_pairs_for_similarity:
        
        similarity_score_total += consine_similarity(a_pair[0], a_pair[1])

    return similarity_score_total/len(word_pairs_for_similarity) #Average similarity score


def normalize_array(an_array):
  
  for i,coord in enumerate(an_array):
      an_array[i] = (an_array[i] - an_array.min()) / (an_array.max() - an_array.min())

  return an_array

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
    )->list:
    """
    A method to preproces text
    """
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text

In [5]:
faq_df = pd.read_csv('FAQs.csv')
faq_df_test = pd.read_csv('FAQs_test.csv')

A Chatbot that similarity search needs to add similiar questions or words. Otherwise, any variation from the original question will not trigger that question.

In [6]:
faq_df

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


So, we add Similar questions or sentences of faq (SQs). This will require human intervention. For example, the first question has a word "When was Albert Einstein born?" I will write the question as "The date of Albert's Einstein's birth", "Albert Einstein's birth date", "Do you the day Albert Einstein's born?". By adding these similar questions, we improve the knowledge base and make the corpus much more searchable.

In [7]:
faq_df['Answer'][0]

'Albert Einstein was born on 14 March 1879.'

In [8]:
faq_df['sq_words'] = 0

In [9]:
faq_df['sq_words'][0] = ["What is the date of his birth", "Do you know the time of his birth", "Tell me the day Albert Einstein was born"]

faq_df['sq_words'][1] = ["Where is the place he was born", "Do you know the birthplace of Albert Einstein", "In what country he was born?"]

faq_df['sq_words'][2] = ["The time of his death", "His date of death", "When wsa the day Albert Einstein died", 'When did Albert Einstein die', 'When dis his life come to an end', 'When did he decease']

faq_df['sq_words'][3] = ["Who is father", "Who is his mother?", "What is the name of his father and mother"]

faq_df['sq_words'][4] = ['Who are his siblings', 'Tell me about his sib', 'Who are the kin of Albert Einstein', 'Who are his relatives']

faq_df['sq_words'][5] = ['The date of his marriage', 'With whom did he have children', 'What is the name of his wife', 'What is the name of his woman']

faq_df['sq_words'][6] = ['What did he study in his university', ' From which university did he graduate', ' From which university did he receive his certificate in physics']

faq_df['sq_words'][7] = ['When did he get a nobel prize', 'What was his award for his contribution', 'What the date he received his nobel prize']

faq_df['sq_words'][8] = ['Why could he not participate in the Nobel Prize ceremony', 'Why could he not come to nobel award show']

faq_df['sq_words'][9] = ['For what reason he got nobel prize?', 'What was his accomplishment in the field of physics', 'What did he do to get nobel prize', 'What did contributed in physics?']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
faq_df['sq_words'][0]

['What is the date of his birth',
 'Do you know the time of his birth',
 'Tell me the day Albert Einstein was born']

In [11]:
faq_df.explode('sq_words')

,Question,Answer,sq_words
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,What is the date of his birth
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,Do you know the time of his birth
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,Tell me the day Albert Einstein was born
1,Where was he born?,"He was born in Ulm, Germany.",Where is the place he was born
1,Where was he born?,"He was born in Ulm, Germany.",Do you know the birthplace of Albert Einstein
1,Where was he born?,"He was born in Ulm, Germany.",In what country he was born?
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",The time of his death
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",His date of death
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",When wsa the day Albert Einstein died
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",When did Albert Einstein die


In [12]:
for i in range(len(faq_df)):
    faq_df['sq_words'][i].append(faq_df['Question'][i])

Creating a corpus from the SQs is an ideal way to map the question to the answer.

In [13]:
faq_df_corpus = faq_df.explode('sq_words')

In [14]:
corpus = faq_df_corpus['sq_words'].to_list()

In [15]:
corpus

['What is the date of his birth',
 'Do you know the time of his birth',
 'Tell me the day Albert Einstein was born',
 'When was Albert Einstein born?',
 'Where is the place he was born',
 'Do you know the birthplace of Albert Einstein',
 'In what country he was born?',
 'Where was he born?',
 'The time of his death',
 'His date of death',
 'When wsa the day Albert Einstein died',
 'When did Albert Einstein die',
 'When dis his life come to an end',
 'When did he decease',
 'When did he die?',
 'Who is father',
 'Who is his mother?',
 'What is the name of his father and mother',
 'Who were his parents?',
 'Who are his siblings',
 'Tell me about his sib',
 'Who are the kin of Albert Einstein',
 'Who are his relatives',
 'Did he have any sisters and brothers?',
 'The date of his marriage',
 'With whom did he have children',
 'What is the name of his wife',
 'What is the name of his woman',
 'Did he marry and have children?',
 'What did he study in his university',
 ' From which university

In [57]:
#Hperparameters
window = 6 #This is the window size between the focus word and the context word in sentence 

Text processing is a crucial part of 

In [58]:
clean_text = []
for sentence in corpus:
   clean_text.append(text_preprocessing(sentence))

In [59]:
clean_text

[['date', 'birth'],
 ['know', 'time', 'birth'],
 ['tell', 'day', 'albert', 'einstein', 'born'],
 ['albert', 'einstein', 'born'],
 ['place', 'born'],
 ['know', 'birthplace', 'albert', 'einstein'],
 ['country', 'born'],
 ['born'],
 ['time', 'death'],
 ['date', 'death'],
 ['wsa', 'day', 'albert', 'einstein', 'died'],
 ['albert', 'einstein', 'die'],
 ['dis', 'life', 'come', 'end'],
 ['decease'],
 ['die'],
 ['father'],
 ['mother'],
 ['name', 'father', 'mother'],
 ['parents'],
 ['siblings'],
 ['tell', 'sib'],
 ['kin', 'albert', 'einstein'],
 ['relatives'],
 ['sisters', 'brothers'],
 ['date', 'marriage'],
 ['children'],
 ['name', 'wife'],
 ['name', 'woman'],
 ['marry', 'children'],
 ['study', 'university'],
 ['university', 'graduate'],
 ['university', 'receive', 'certificate', 'physics'],
 ['receive', 'education'],
 ['get', 'nobel', 'prize'],
 ['award', 'contribution'],
 ['date', 'received', 'nobel', 'prize'],
 ['albert', 'einstein', 'awarded', 'nobel', 'prize', 'physics'],
 ['could', 'partic

In [60]:
word_list = []
all_text = []

In [61]:
for text in corpus:

    # Cleaning the text
    text = text_preprocessing(text)

    # Appending to the all text list
    all_text += text #This will be used for creating a unique word dictionary

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_list.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_list.append([word] + [text[(i - w - 1)]])

In [62]:
word_list

[['date', 'birth'],
 ['birth', 'date'],
 ['know', 'time'],
 ['know', 'birth'],
 ['time', 'birth'],
 ['time', 'know'],
 ['birth', 'time'],
 ['birth', 'know'],
 ['tell', 'day'],
 ['tell', 'albert'],
 ['tell', 'einstein'],
 ['tell', 'born'],
 ['day', 'albert'],
 ['day', 'tell'],
 ['day', 'einstein'],
 ['day', 'born'],
 ['albert', 'einstein'],
 ['albert', 'day'],
 ['albert', 'born'],
 ['albert', 'tell'],
 ['einstein', 'born'],
 ['einstein', 'albert'],
 ['einstein', 'day'],
 ['einstein', 'tell'],
 ['born', 'einstein'],
 ['born', 'albert'],
 ['born', 'day'],
 ['born', 'tell'],
 ['albert', 'einstein'],
 ['albert', 'born'],
 ['einstein', 'born'],
 ['einstein', 'albert'],
 ['born', 'einstein'],
 ['born', 'albert'],
 ['place', 'born'],
 ['born', 'place'],
 ['know', 'birthplace'],
 ['know', 'albert'],
 ['know', 'einstein'],
 ['birthplace', 'albert'],
 ['birthplace', 'know'],
 ['birthplace', 'einstein'],
 ['albert', 'einstein'],
 ['albert', 'birthplace'],
 ['albert', 'know'],
 ['einstein', 'albert

In [63]:
words = list(set(all_text))

In [64]:
words.sort()

In [65]:
words

['accomplishment',
 'albert',
 'attend',
 'award',
 'awarded',
 'birth',
 'birthplace',
 'born',
 'brothers',
 'ceremony',
 'certificate',
 'children',
 'come',
 'contributed',
 'contribution',
 'could',
 'country',
 'date',
 'day',
 'death',
 'decease',
 'die',
 'died',
 'dis',
 'education',
 'einstein',
 'end',
 'father',
 'field',
 'get',
 'got',
 'graduate',
 'kin',
 'know',
 'life',
 'marriage',
 'marry',
 'mother',
 'name',
 'nobel',
 'parents',
 'participate',
 'physics',
 'place',
 'prize',
 'reason',
 'receive',
 'received',
 'relatives',
 'show',
 'sib',
 'siblings',
 'sisters',
 'study',
 'tell',
 'time',
 'university',
 'wife',
 'woman',
 'wsa']

In [66]:
albert_dictionary = {}

for i, word in enumerate(words):
    albert_dictionary.update({word: i})

In [67]:
albert_dictionary

{'accomplishment': 0,
 'albert': 1,
 'attend': 2,
 'award': 3,
 'awarded': 4,
 'birth': 5,
 'birthplace': 6,
 'born': 7,
 'brothers': 8,
 'ceremony': 9,
 'certificate': 10,
 'children': 11,
 'come': 12,
 'contributed': 13,
 'contribution': 14,
 'could': 15,
 'country': 16,
 'date': 17,
 'day': 18,
 'death': 19,
 'decease': 20,
 'die': 21,
 'died': 22,
 'dis': 23,
 'education': 24,
 'einstein': 25,
 'end': 26,
 'father': 27,
 'field': 28,
 'get': 29,
 'got': 30,
 'graduate': 31,
 'kin': 32,
 'know': 33,
 'life': 34,
 'marriage': 35,
 'marry': 36,
 'mother': 37,
 'name': 38,
 'nobel': 39,
 'parents': 40,
 'participate': 41,
 'physics': 42,
 'place': 43,
 'prize': 44,
 'reason': 45,
 'receive': 46,
 'received': 47,
 'relatives': 48,
 'show': 49,
 'sib': 50,
 'siblings': 51,
 'sisters': 52,
 'study': 53,
 'tell': 54,
 'time': 55,
 'university': 56,
 'wife': 57,
 'woman': 58,
 'wsa': 59}

In [68]:
total_albert_words = len(albert_dictionary)
albert_words = list(albert_dictionary.keys())

total_words, words

In [69]:
n_focused_encoding = []
m_context_encoding = []

X_focused = []
Y_context = []

In [70]:
for i, word_list in enumerate(word_list):

    focused_row = np.zeros(total_albert_words)
    context_row = np.zeros(total_albert_words)
    
    focus_word_index = albert_dictionary.get(word_list[0])
    context_word_index = albert_dictionary.get(word_list[1])

    focused_row[focus_word_index] = 1
    context_row[context_word_index] = 1

    n_focused_encoding.append(focused_row)
    m_context_encoding.append(context_row)

    
X_focused = np.array(n_focused_encoding)
Y_context = np.array(m_context_encoding)
    

In [71]:
X_focused

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
Y_context

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
X_focused.shape

(298, 60)

In [74]:
Y_context.shape

(298, 60)

In [91]:
#Hyperparameter
embed_size = 3 

In [92]:
input_layer = tf.keras.Input(shape=X_focused.shape[1],)

x_ = tf.keras.layers.Dense(units=embed_size)(input_layer)
output_layer = tf.keras.layers.Dense(units=Y_context.shape[1], activation=tf.nn.softmax)(x_)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy') #optimizer and loss function

Time for some training!!!

In [93]:
model.fit(
    x=X_focused, 
    y=Y_context, 
    batch_size=5,
    epochs=3000 
    )

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step - loss: 3.1774
Epoch 502/3000
1/1 [==============================] - 0s 10ms/step - loss: 3.1762
Epoch 503/3000
1/1 [==============================] - 0s 12ms/step - loss: 3.1751
Epoch 504/3000
1/1 [==============================] - 0s 12ms/step - loss: 3.1740
Epoch 505/3000
1/1 [==============================] - 0s 14ms/step - loss: 3.1728
Epoch 506/3000
1/1 [==============================] - 0s 13ms/step - loss: 3.1717
Epoch 507/3000
1/1 [==============================] - 0s 18ms/step - loss: 3.1705
Epoch 508/3000
1/1 [==============================] - 0s 12ms/step - loss: 3.1694
Epoch 509/3000
1/1 [==============================] - 0s 10ms/step - loss: 3.1682
Epoch 510/3000
1/1 [==============================] - 0s 18ms/step - loss: 3.1671
Epoch 511/3000
1/1 [==============================] - 0s 13ms/step - loss: 3.1659
Epoch 512/3000
1/1 [==============================] - 0s 14ms/

Loss seems good given the lack of proper computing power. The loss can be decreased given the hyparameters can be changed, which requires a high-end computational power.

In [94]:
albert_weights = model.get_weights()[0]

Embeddings of a word. Need more computing power for

In [95]:
albert_weights

array([[-0.5714815 , -3.081645  , -0.6178899 ],
       [ 0.485748  ,  0.45087057,  0.4446155 ],
       [-1.7205558 ,  1.374151  ,  1.6060913 ],
       [-1.209305  ,  0.16802512,  1.0308747 ],
       [-0.43089023,  0.3686658 , -0.01066851],
       [ 2.0045485 ,  0.88418275, -1.7761023 ],
       [ 0.841571  ,  2.100633  ,  0.21935672],
       [ 1.0415896 ,  1.6754214 ,  1.3530545 ],
       [ 3.069484  , -0.50395286, -3.3435087 ],
       [-2.0056813 ,  1.129861  ,  1.3712357 ],
       [-2.3607733 , -1.7425663 , -1.8544712 ],
       [ 2.4168086 , -0.7338898 , -1.3393309 ],
       [-0.47272503, -0.8060853 ,  1.27962   ],
       [-2.3650773 , -2.4337184 , -0.7491953 ],
       [-1.7970103 ,  0.49829623,  2.8947701 ],
       [-2.6586275 ,  0.28985155,  1.1885179 ],
       [ 1.7824557 , -0.46420354,  2.867806  ],
       [-1.1125126 ,  2.882203  , -1.900357  ],
       [ 0.8206871 ,  2.2787313 ,  2.2174113 ],
       [ 1.7202419 ,  0.9308089 , -2.3751035 ],
       [ 0.1716392 ,  0.16988811,  0.282

In [96]:
#Creating an embedding dictionary for now

word_ebmedding_dict = {}

for word in words:
    word_ebmedding_dict.update({
        word: normalize_array(albert_weights[albert_dictionary.get(word)])
    })

In [97]:
for a_word_embedding in word_ebmedding_dict:
  print(a_word_embedding)

accomplishment
albert
attend
award
awarded
birth
birthplace
born
brothers
ceremony
certificate
children
come
contributed
contribution
could
country
date
day
death
decease
die
died
dis
education
einstein
end
father
field
get
got
graduate
kin
know
life
marriage
marry
mother
name
nobel
parents
participate
physics
place
prize
reason
receive
received
relatives
show
sib
siblings
sisters
study
tell
time
university
wife
woman
wsa


In [106]:
faq_df_corpus['ranking'] = 0
testing_question = faq_df_test['Question'][1] #Change the index number (0~4) for testing all the questions

In [107]:
faq_df_corpus.columns

Index(['Question', 'Answer', 'sq_words', 'ranking'], dtype='object')

In [108]:
faq_df_corpus['ranking'] = faq_df_corpus['sq_words'].apply(lambda a_sentence: sentence_similarity(a_sentence, testing_question))

In [109]:
faq_df_corpus

,Question,Answer,sq_words,ranking
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,What is the date of his birth,0.465645
1,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,Do you know the time of his birth,0.389950
2,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,Tell me the day Albert Einstein was born,0.466119
3,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.,When was Albert Einstein born?,0.291658
4,Where was he born?,"He was born in Ulm, Germany.",Where is the place he was born,0.494730
5,Where was he born?,"He was born in Ulm, Germany.",Do you know the birthplace of Albert Einstein,0.315476
6,Where was he born?,"He was born in Ulm, Germany.",In what country he was born?,0.526571
7,Where was he born?,"He was born in Ulm, Germany.",Where was he born?,0.748469
8,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",The time of his death,0.304783
9,When did he die?,"He died 18 April 1955 in Princeton, New Jersey...",His date of death,0.468622


In [110]:
faq_df_corpus.reset_index(inplace=True)

In [111]:
faq_df_corpus.drop(columns=['index'], axis = 1, inplace=True)

Testing Questions

In [104]:
testing_question

'What is the date of his death?'

Answer to the question

In [105]:
faq_df_corpus.loc[faq_df_corpus['ranking'].idxmax()]['Answer']

'He had one sister named Maja.'

Currently, the model is not able to give a proper answer (also known as wrong-mapping in an E-commerce context). This can be overcome by adding more SQs and enhancing the corpus and training the model several times.

Also, the current model does not take into account about data augmentation and spell correction for not only English language but also Roman Bangla letters. So, there is much more work that needs to be done.